In [1]:
# this notebook requires plato and vispy for visualization
import numpy as np
from numpy import linalg as LA
from fedorov import AflowPrototype, convert_to_box, fractional_to_cartesian
import plato

import vispy, vispy.app
vispy.app.use_app('ipynb_webgl')
import plato.draw.vispy as draw

from ipywidgets import interactive, interact
import ipywidgets as widgets
import IPython
import os
import pandas as pd


In [2]:
def make_positions_for_box(basis_vectors, type_list):
    basis_vectors = np.vstack((basis_vectors, basis_vectors + [0,0,1], basis_vectors + [0,1,0],
                               basis_vectors + [1,0,0], basis_vectors + [0,1,1], basis_vectors + [1,0,1],
                               basis_vectors + [1,1,0], basis_vectors + [1,1,1]))
    type_list = type_list * 8
    N = len(type_list)
    index = np.arange(0,N,1)
    index = index[LA.norm(basis_vectors, ord=np.inf, axis=1)<1.001].tolist()
    type_list = [type_list[i] for i in index]
    basis_vectors = basis_vectors[LA.norm(basis_vectors, ord=np.inf, axis=1)<1.001,:]
    basis_vectors -= [0.5,0.5,0.5]
    return  basis_vectors, type_list

In [3]:
# Color from
# http://www.randalolson.com/2014/06/28/how-to-make-beautiful-data-visualizations-in-python-with-matplotlib/   
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]    
 
for i in range(len(tableau20)):    
    r, g, b = tableau20[i]    
    tableau20[i] = (r / 255., g / 255., b / 255., 1)

type_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T']

colormap = dict(zip(type_list, tableau20))

In [6]:
Aflow_data_dir = '../fedorov/crystal_data/Aflow_processed_data.csv'
Aflow_database = pd.read_csv(Aflow_data_dir, index_col=0)
prototype_list = dict(zip(Aflow_database.index, Aflow_database['id']))
Aflow_database.head()

,id,Pearson_symbol,space_group_number,Wyckoff_site,lattice_params_list,basis_params_list,lattice_params_value_list,basis_params_value_list
prototype_index,,,,,,,,
0,hP4-C-194,hP4,194,['f'],"['a', 'c/a']",['z1'],"[2.508, 1.66786283892]",[0.05995]
1,cP8-Cr3Si-223,cP8,223,"['c', 'a']",['a'],[],[4.556],[]
2,tI18-V4Zn5-139,tI18,139,"['i', 'ah']","['a', 'c/a']","['x2', 'x3']","[8.91, 0.361391694725]","[0.328, 0.348]"
3,tI28-Mn3O4-141,tI28,141,"['ad', 'h']","['a', 'c/a']","['y3', 'z3']","[5.765, 1.63781439722]","[0.0278, 0.2589]"
4,hP14-B5W2-194,hP14,194,"['abdf', 'f']","['a', 'c/a']","['z4', 'z5']","[2.982, 4.651240778]","[0.528, 0.139]"


In [7]:
def visualize(prototype_index):
    print(prototype_list[prototype_index])
    
    new_structure = AflowPrototype(prototype_index=prototype_index, print_info=True, set_type=True)
    basis_vectors, type_list = new_structure.get_basis_vectors()
    lattice_vectors = new_structure.get_lattice_vectors()
    basis_vectors, type_list= make_positions_for_box(basis_vectors, type_list)
    Lx, Ly, Lz, xy, xz, yz = convert_to_box(lattice_vectors)
    scaled_basis = fractional_to_cartesian(basis_vectors, lattice_vectors)
    
    N = len(scaled_basis)
    colors = [colormap[type] for type in type_list]
    radii = [0.5]*N
    
    particle_prim = draw.Spheres(positions=scaled_basis, colors=colors, radii=radii)
    box_prim = draw.Box(Lx=Lx, Ly=Ly, Lz=Lz, xy=xy, xz=xz, yz=yz, width=0.05)
    scene = draw.Scene([particle_prim, box_prim], size=[30,30])
    scene.show()

In [8]:
visualize(133)

oP8-Np-62
Info for the chosen crystal structure prototype:
 id: oP8-Np-62, (Pearson-Chemistry-SpaceGroup)
 Wyckoff sites: ['cc']
 available lattice parameters: {'a': 6.663, 'b': 4.722999999999612, 'c': 4.88700000000231}
 available basis parameters: {'x1': 0.464, 'z1': 0.292, 'x2': 0.181, 'z2': 0.658}


VispyWidget(height=600, width=600)